In [1]:
import os
print(os.getcwd())


/Users/maceli/ifood_cs/code/notebook


In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('..')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('future.no_silent_downcasting', True)


import utilities.functions as functions
import utilities.descritiva as descritiva

from utilities.functions import (
    load_data,
    check_key_uniqueness,
    merge_df,
    load_orders,
    process_orders_pandas,
)
from utilities.descritiva import (
    plot_indicadores_mes_plataforma,
    gerar_stats,
    pedidos_group,
    criacao_ordens,
    conversao_imediata,
teste_proporcao_por_janela,
teste_ab_completo_por_janela,
    analisar_ab_completo,
  verificar_poder_janelas,
  calcular_mde_por_janela
)

In [2]:
from pathlib import Path
BASE_PATH = Path("/Users/maceli/ifood_cs/dados") 
df = pd.read_parquet(BASE_PATH / "silver" / "df_publico_orders.parquet")#.drop([ "delivery_address_district", "delivery_address_country",'customer_name','active','created_at','delivery_address_external_id'], axis=1)
df.head()

,customer_id,is_target,active,created_at,delivery_address_state,merchant_id,order_created_at,order_id,order_total_amount,origin_platform
0,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,target,True,2018-04-06T02:48:42.887Z,SP,e80a8d7a13b52ca9ccf247ca3396f1018be41a5b014d79...,2019-01-11T14:07:48.000Z,c9a236fc0b0255433d0852dc90ef8b9949f3a3ab67e553...,10.0,ANDROID
1,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,target,True,2018-04-06T02:48:42.887Z,SP,05291eacc3cb88847b56af15838a0e653a1fcda54a1d88...,2019-01-05T13:38:28.000Z,db3b910b374ce240d551b829b297b9547f4e2feee9fd78...,12.0,ANDROID
2,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,target,True,2018-04-06T02:48:42.887Z,SP,05291eacc3cb88847b56af15838a0e653a1fcda54a1d88...,2019-01-13T13:50:06.000Z,a20903719abdaae9f6639681845f95963de83e0047ff13...,13.0,ANDROID
3,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,target,True,2018-04-06T02:48:42.887Z,SP,05291eacc3cb88847b56af15838a0e653a1fcda54a1d88...,2019-01-26T13:15:53.000Z,90652cc196fbfa352b1d4d61b18b5babf39fad949c7ccc...,13.0,ANDROID
4,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,target,True,2018-04-06T02:48:42.887Z,SP,05291eacc3cb88847b56af15838a0e653a1fcda54a1d88...,2019-01-15T13:48:33.000Z,9091a0ead77e2ed0d5ed9ce6787631386bd120036ff5b7...,13.0,ANDROID


id_sample=['000009e29b12d8c5b9c9579e21384320c2df3bd935f19e12402be6b07a099361',
       '000021924bf8192f64a7cc17ce7c0bc511cba907308f80f3be7d81df1480e909',
       '000022ced2f3aa411b66ac41167b272ae528c4acdffeebb88a59af9ceeba4717',
       '000032b594523c3f8868edee4f1577b157e115cd01ab3110b378b06843dafd77',
       '0000bb10fb47a1d6b2d73754ef383950ef536c77d07212e431c3ff77f68834c9',
       '0000c21984ae00cefb5d4931bfa49483dde546413c9b40c4228220f27d7ecdf2',
       '00010814ab1e602ccbc9ffe3b64f44973491b2fd6d8ab7d19b138f09cdc040d0',
       '00010e03ea7d7c392a9b6cabcbed034e64fb54214e90bf89075af3d87e744023',
       '000116a7bc0a1da0c86ba2dddc02fcc70d35f8b900d69305d7eb13757f8a0c44',
       '0001226e5175177581c0e520732ba58a61dfa96d57be0caef75f5bc948887a8c']
df = df[df['customer_id'].isin(id_sample)]

In [4]:
#ids_sample = df['customer_id'].sample(n=100, random_state=42)
#df = df[df['customer_id'].isin(ids_sample)]

In [3]:
df["order_created_at"] = pd.to_datetime(df["order_created_at"], utc=True)
df['unique_order_hash'] = pd.util.hash_pandas_object(
        df[['customer_id', 'order_created_at','order_id']],
        index=False
    )
df = df.drop(columns=['order_id'], errors='ignore')
df["order_created_month"] = df["order_created_at"].dt.month
id_both_monht=df[df['order_created_month']==12]['customer_id'].unique()
df = df[df['customer_id'].isin(id_both_monht)].reset_index(drop=True)
df["day_month_year"] = df["order_created_at"].dt.strftime("%d-%m-%Y")
df["weekday"] = df["order_created_at"].dt.day_name()
df["hour"] = df["order_created_at"].dt.hour

df["day"] = df["order_created_at"].dt.day


In [4]:
df, dias_0=criacao_ordens(df,group_vars='is_target')
print('Media da diferenca em dias entre a primeira e segunda ordem, nao levando em consideracao o periodo do mes',dias_0)

Media da diferenca em dias entre a primeira e segunda ordem, nao levando em consideracao o periodo do mes 5.802379096308715


Dias entre a primeira ordem e segunda

In [7]:
summary_mes = gerar_stats(
    df[df['rank_month'] == 2],
    ['days_since_first_order_month']
)

#summary_mes.head(10)

In [7]:
janelas = [(0,30),(0, 1), (0, 3), (0, 4),
           (5,6), (10,15),
            (10, 16)]

df_base_final = None
todos_resumos = []

for window_start, window_end in janelas:

    resumo, df_p = conversao_imediata(
        df,
        mes_base=12,
        r_ordem=2,
        window_start=window_start,
        window_end=window_end,
        fill_value=-999
    )

    # guardar resumo de cada iteração
    todos_resumos.append(resumo)

    # nome da coluna que a função CRIA
    col_origem = f"converteu{window_start}-{window_end}d"
    # nome da coluna que você quer na base final
    col_destino = f"converteu_{window_start}_{window_end}"

    # cria a nova coluna usando a coluna dinâmica da função
    df_p[col_destino] = df_p[col_origem]

    # montar/atualizar base final mantendo target e valor
    if df_base_final is None:
        df_base_final = df_p[
            ['customer_id', 'is_target', 'order_total_amount', col_destino]
        ].copy()
    else:
        df_base_final = df_base_final.merge(
            df_p[['customer_id', col_destino]],
            on='customer_id',
            how='left'
        )

# preencher NaN das flags com 0
for window_start, window_end in janelas:
    col_destino = f"converteu_{window_start}_{window_end}"
    df_base_final[col_destino] = df_base_final[col_destino].fillna(0).astype(int)

# resumo com TODAS as iterações
df_resumo_final = pd.concat(todos_resumos, ignore_index=True)


In [9]:
df_resumo_final.columns#

Index(['is_target', 'total_clientes', 'clientes_convertidos', 'total_amount',
       'total_amount_convertido', 'taxa_conversao', 'window_range',
       'mes_base'],
      dtype='object')

In [8]:
teste_proporcao_por_janela(df_base_final)

,janela,total_control,conv_control,total_target,conv_target,taxa_control,taxa_target,z_stat,p_value,significativo_5%
0,converteu_0_30,241457,101857,312919,152107,42.18,48.61,47.604617,0.000000e+00,True
1,converteu_0_1,241457,28418,312919,42430,11.77,13.56,19.793622,3.378667e-87,True
2,converteu_0_3,241457,45195,312919,67379,18.72,21.53,25.831601,3.916965e-147,True
3,converteu_0_4,241457,51524,312919,77110,21.34,24.64,28.889781,1.604590e-183,True
4,converteu_5_6,241457,12570,312919,18578,5.21,5.94,11.720781,9.974832e-32,True
5,converteu_10_15,241457,15240,312919,22874,6.31,7.31,14.563863,4.768496e-48,True
6,converteu_10_16,241457,16619,312919,24904,6.88,7.96,15.088042,1.941183e-51,True


In [9]:
df_resultados=teste_ab_completo_por_janela(df_base_final)

In [10]:
df_resultados

,janela,total_control,conv_control,total_target,conv_target,taxa_control_%,taxa_target_%,diferenca_taxa_%,z_stat,p_value_proporcao,...,ic_sup_media,mw_stat,mw_p_value,decisao_mw,t_welch_stat,t_welch_p_value,decisao_welch,t_normal_stat,t_normal_p_value,decisao_t_normal
0,converteu_0_30,241457,101857,312919,152107,42.1843,48.6091,6.4247,47.604617,0.000000e+00,...,1.388455,7.756023e+09,0.602083,Não rejeitar H0,-0.947462,0.343406,Não rejeitar H0,-1.154920,0.248124,Não rejeitar H0
1,converteu_0_1,241457,28418,312919,42430,11.7694,13.5594,1.7900,19.793622,3.378667e-87,...,4.203707,6.024558e+08,0.871365,Não rejeitar H0,-1.101504,0.270687,Não rejeitar H0,-1.345044,0.178615,Não rejeitar H0
2,converteu_0_3,241457,45195,312919,67379,18.7176,21.5324,2.8148,25.831601,3.916965e-147,...,2.602365,1.519054e+09,0.507454,Não rejeitar H0,-1.115455,0.264661,Não rejeitar H0,-1.360518,0.173669,Não rejeitar H0
3,converteu_0_4,241457,51524,312919,77110,21.3388,24.6422,3.3034,28.889781,1.604590e-183,...,2.348866,1.985664e+09,0.897140,Não rejeitar H0,-1.091248,0.275169,Não rejeitar H0,-1.333299,0.182436,Não rejeitar H0
4,converteu_5_6,241457,12570,312919,18578,5.2059,5.9370,0.7311,11.720781,9.974832e-32,...,0.821125,1.163511e+08,0.597004,Não rejeitar H0,0.131402,0.895458,Não rejeitar H0,0.130751,0.895973,Não rejeitar H0
5,converteu_10_15,241457,15240,312919,22874,6.3117,7.3099,0.9982,14.563863,4.768496e-48,...,1.338707,1.750436e+08,0.479681,Não rejeitar H0,1.675621,0.093822,Não rejeitar H0,1.670531,0.094823,Não rejeitar H0
6,converteu_10_16,241457,16619,312919,24904,6.8828,7.9586,1.0758,15.088042,1.941183e-51,...,1.126915,2.074529e+08,0.668092,Não rejeitar H0,1.263145,0.206545,Não rejeitar H0,1.261248,0.207227,Não rejeitar H0


In [11]:
import os

if os.path.exists('../Resultados'):
    df_resultados.to_csv('../Resultados/df_resultados.csv', index=False)
else:
    # Criar pasta ou salvar em outro lugar
    os.makedirs('../Resultados', exist_ok=True)
    df_resultados.to_csv('../Resultados/df_resultados.csv', index=False)
    print("Pasta criada e arquivo salvo!")

In [12]:
df_poder = verificar_poder_janelas(df_resultados)
df_poder

,janela,cohen_d,amostra_target,amostra_control,poder_estatistico
0,converteu_0_30,-0.004676,312919,241457,0.456114
1,converteu_0_1,-0.010310,312919,241457,0.982922
2,converteu_0_3,-0.008272,312919,241457,0.905250
3,converteu_0_4,-0.007587,312919,241457,0.851038
4,converteu_5_6,0.001510,312919,241457,0.091769
5,converteu_10_15,0.017468,312919,241457,1.000000
6,converteu_10_16,0.012633,312919,241457,0.998805


In [13]:
calcular_mde_por_janela(df_base_final)

,janela,mde_reais,mde_effect_size,effect_size_real,amostra_target,amostra_control,std_pooled
0,converteu_0_30,3.151832,0.011343,-0.004676,152107,101857,277.859883
1,converteu_0_1,11.234543,0.021475,-0.010310,42430,28418,523.135795
2,converteu_0_3,7.077871,0.017034,-0.008272,67379,45195,415.517902
3,converteu_0_4,6.200552,0.015943,-0.007587,77110,51524,388.910292
4,converteu_5_6,1.105448,0.032357,0.001510,18578,12570,34.164395
5,converteu_10_15,1.034760,0.029295,0.017468,22874,15240,35.322083
6,converteu_10_16,0.980979,0.028061,0.012633,24904,16619,34.958585


In [15]:
df_base_final.head()

,customer_id,is_target,order_total_amount,converteu_0_30,converteu_0_1,converteu_0_3,converteu_0_4,converteu_10_16
0,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,83.50,1,0,0,0,0
1,000021924bf8192f64a7cc17ce7c0bc511cba907308f80...,target,63.00,1,0,0,0,0
2,000022ced2f3aa411b66ac41167b272ae528c4acdffeeb...,target,18.95,1,0,0,0,0
3,000032b594523c3f8868edee4f1577b157e115cd01ab31...,control,0.00,0,0,0,0,0
4,0000bb10fb47a1d6b2d73754ef383950ef536c77d07212...,control,84.20,1,1,1,1,0


In [17]:
def aplicar_desconto_todas_flags(df, descontos_percentuais=[0]):
    """
    Aplica diferentes percentuais de desconto em todas as flags converteu_*
    """
    df_result = df.copy()
    
    # Encontra automaticamente todas as colunas que começam com "converteu_"
    colunas_flags = [col for col in df.columns if col.startswith('converteu_')]
    
    print(f"🎯 Aplicando descontos {descontos_percentuais}% nas flags: {colunas_flags}")
    
    for desconto in descontos_percentuais:
        for col_flag in colunas_flags:
            # Aplica desconto para cada flag
            mask = (df_result[col_flag] == 1) & (df_result['is_target'] == 'target')
            nova_coluna = f'order_total_amount_{col_flag}_desconto_{desconto}%'
            
            # Inicializa a coluna com 0
            df_result[nova_coluna] = 0
            
            # Aplica desconto apenas onde a condição é verdadeira
            df_result.loc[mask, nova_coluna] = (
                df_result.loc[mask, 'order_total_amount'] * desconto/100
            )
    
    return df_result

# Uso com diferentes descontos
df_com_descontos = aplicar_desconto_todas_flags(df_base_final, descontos_percentuais=[5, 10, 15, 20,25]
                                                )

# Ou com descontos específicos


🎯 Aplicando descontos [5, 10, 15, 20, 25]% nas flags: ['converteu_0_30', 'converteu_0_1', 'converteu_0_3', 'converteu_0_4', 'converteu_5_6', 'converteu_10_15', 'converteu_10_16']


In [17]:
df_com_descontos[df_com_descontos['converteu_10_16']==1]

,customer_id,is_target,order_total_amount,converteu_0_30,converteu_0_1,converteu_0_3,converteu_0_4,converteu_10_16,order_total_amount_converteu_0_30_desconto_5%,order_total_amount_converteu_0_1_desconto_5%,...,order_total_amount_converteu_0_30_desconto_20%,order_total_amount_converteu_0_1_desconto_20%,order_total_amount_converteu_0_3_desconto_20%,order_total_amount_converteu_0_4_desconto_20%,order_total_amount_converteu_10_16_desconto_20%,order_total_amount_converteu_0_30_desconto_25%,order_total_amount_converteu_0_1_desconto_25%,order_total_amount_converteu_0_3_desconto_25%,order_total_amount_converteu_0_4_desconto_25%,order_total_amount_converteu_10_16_desconto_25%
8,000116a7bc0a1da0c86ba2dddc02fcc70d35f8b900d693...,target,14.00,1,0,0,0,1,0.700,0.0,...,2.80,0.0,0.0,0.0,2.80,3.500,0.0,0.0,0.0,3.500
15,00021cd56b6d6c980c3b48ca0fdf8c53cf3fea776d74ab...,target,13.90,1,0,0,0,1,0.695,0.0,...,2.78,0.0,0.0,0.0,2.78,3.475,0.0,0.0,0.0,3.475
24,00030dc409e20c495dab3036d6b16813b3ea2390ab9897...,control,35.01,1,0,0,0,1,0.000,0.0,...,0.00,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.000
34,00044a499977a84f70cc8059efdc6d0bfec20b1cb97fc4...,control,31.90,1,0,0,0,1,0.000,0.0,...,0.00,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.000
40,00053524223cc350dff687a3f9d0db2bf64c9505871548...,control,51.00,1,0,0,0,1,0.000,0.0,...,0.00,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554275,fff67096f01bbd4d54de5d520a5800b496eb94cd53444e...,target,30.70,1,0,0,0,1,1.535,0.0,...,6.14,0.0,0.0,0.0,6.14,7.675,0.0,0.0,0.0,7.675
554285,fff7918fe1b93c7311f94a944ad9a1f1392ef1c2242064...,control,29.90,1,0,0,0,1,0.000,0.0,...,0.00,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.000
554307,fff9a66766215f2f740bf0cba137d29745d64924a5aeb5...,target,49.00,1,0,0,0,1,2.450,0.0,...,9.80,0.0,0.0,0.0,9.80,12.250,0.0,0.0,0.0,12.250
554335,fffbe36900db92352e3d92ea6f632511080a1053d9dec0...,control,21.50,1,0,0,0,1,0.000,0.0,...,0.00,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.000


In [18]:
def calcular_viabilidade_financeira_automacao(df, descontos_percentuais=[5, 10, 15, 20,25]):
    """
    Soma os descontos das colunas já existentes
    """
    resultados = []
    colunas_flags = [col for col in df.columns if col.startswith('converteu_')]
    
    for desconto in descontos_percentuais:
        for col_flag in colunas_flags:
            # Nome da coluna com desconto
           
            coluna_desconto = f'order_total_amount_{col_flag}_desconto_{desconto}%'
            print(coluna_desconto)
            # Filtra os dados
            target_data = df[(df[col_flag] == 1) & (df['is_target'] == 'target')]
            control_data = df[(df[col_flag] == 1) & (df['is_target'] == 'control')]
            
            # Receitas originais
            receita_target_original = target_data['order_total_amount'].sum()
            receita_control_original = control_data['order_total_amount'].sum()
            
            # Soma do desconto da coluna específica
            total_desconto = target_data[coluna_desconto].sum() if coluna_desconto in df.columns else 0
            ifood_t = receita_target_original * .12
            ifood_td= ifood_t-total_desconto
            ifood_c= receita_control_original * .12
            conversoes_adicionais = len(target_data) - len(control_data)
            custo_por_conversao_adicional = total_desconto / conversoes_adicionais if conversoes_adicionais > 0 else 0

            resultados.append({
                'janela': col_flag,
                'desconto_%': desconto,
                'receita_target': receita_target_original,
                'receita_control': receita_control_original,
                'total_desconto': total_desconto,
                 'ifood_t': ifood_t,
                 "ifood_td":ifood_td,
                 'ifood_c': ifood_c,
                 'conversoes_adicionais': conversoes_adicionais,
                 'custo_por_conversao_adicional':custo_por_conversao_adicional

            })
    
    return pd.DataFrame(resultados)

In [22]:
via_f=calcular_viabilidade_financeira_automacao(df_com_descontos)



order_total_amount_converteu_0_30_desconto_5%
order_total_amount_converteu_0_1_desconto_5%
order_total_amount_converteu_0_3_desconto_5%
order_total_amount_converteu_0_4_desconto_5%
order_total_amount_converteu_5_6_desconto_5%
order_total_amount_converteu_10_15_desconto_5%
order_total_amount_converteu_10_16_desconto_5%
order_total_amount_converteu_0_30_desconto_10%
order_total_amount_converteu_0_1_desconto_10%
order_total_amount_converteu_0_3_desconto_10%
order_total_amount_converteu_0_4_desconto_10%
order_total_amount_converteu_5_6_desconto_10%
order_total_amount_converteu_10_15_desconto_10%
order_total_amount_converteu_10_16_desconto_10%
order_total_amount_converteu_0_30_desconto_15%
order_total_amount_converteu_0_1_desconto_15%
order_total_amount_converteu_0_3_desconto_15%
order_total_amount_converteu_0_4_desconto_15%
order_total_amount_converteu_5_6_desconto_15%
order_total_amount_converteu_10_15_desconto_15%
order_total_amount_converteu_10_16_desconto_15%
order_total_amount_convert

In [26]:
via[via_f['janela']=="converteu_10_15"]

,janela,desconto_%,receita_target,receita_control,total_desconto,ifood_t,ifood_td,ifood_c,conversoes_adicionais,custo_por_conversao_adicional
5,converteu_10_15,5,1114686.25,733266.38,55734.3125,133762.35,78028.0375,87991.9656,7634,7.300801
12,converteu_10_15,10,1114686.25,733266.38,111468.6250,133762.35,22293.7250,87991.9656,7634,14.601601
19,converteu_10_15,15,1114686.25,733266.38,167202.9375,133762.35,-33440.5875,87991.9656,7634,21.902402
26,converteu_10_15,20,1114686.25,733266.38,222937.2500,133762.35,-89174.9000,87991.9656,7634,29.203203
33,converteu_10_15,25,1114686.25,733266.38,278671.5625,133762.35,-144909.2125,87991.9656,7634,36.504003


In [25]:
import os

if os.path.exists('../Resultados'):
    df_resultados.to_csv('../Resultados/via_f.csv', index=False)
else:
    # Criar pasta ou salvar em outro lugar
    os.makedirs('../Resultados', exist_ok=True)
    df_resultados.to_csv('../Resultados/via_f.csv', index=False)
    print("Pasta criada e arquivo salvo!")

In [ ]:
calcular_viabilidade_financeira_automacao2(df_com_descontos)

In [ ]:
def calcular_viabilidade_financeira_automacao2(df, descontos_percentuais=[5], comissao_ifood=0.12):
    resultados = []
    colunas_flags = [col for col in df.columns if col.startswith('converteu_')]
    
    for desconto in descontos_percentuais:
        for col_flag in colunas_flags:
            coluna_desconto = f'order_total_amount_{col_flag}_desconto_{desconto}%'
            
            target_data = df[(df[col_flag] == 1) & (df['is_target'] == 'target')]
            control_data = df[(df[col_flag] == 1) & (df['is_target'] == 'control')]
            
            receita_target_original = target_data['order_total_amount'].sum()
            receita_control_original = control_data['order_total_amount'].sum()
            
            receita_target_com_desconto = target_data[coluna_desconto].sum() if coluna_desconto in df.columns else 0
            
            # iFood recebe 12% do valor ORIGINAL
            ifood_recebe_control = receita_target_original * comissao_ifood
            ifood_recebe_target = receita_control_original * comissao_ifood
            
            # Custo do desconto para o iFood = soma total dos descontos dados
            custo_desconto_ifood = ifood_recebe_target - receita_target_com_desconto
            roi_desconto_ifood = (ifood_recebe_target / custo_desconto_ifood) * 100 if custo_desconto_ifood > 0 else 0
            
            # 3. Eficiência do desconto (custo por conversão adicional)
            conversoes_adicionais = len(target_data) - len(control_data)
            custo_por_conversao_adicional = custo_desconto_ifood / conversoes_adicionais if conversoes_adicionais > 0 else 0
            
            # 4. Ticket médio comparativo
            ticket_medio_target = receita_target_original / len(target_data) if len(target_data) > 0 else 0
            ticket_medio_control = receita_control_original / len(control_data) if len(control_data) > 0 else 0

            resultados.append({
                'janela': col_flag,
                'desconto_%': desconto,
                'receita_target_original': receita_target_original,
                'total desconto': receita_target_com_desconto,
                'receita_control': receita_control_original,
                'ifood_recebe_target': ifood_recebe_target,
                'ifood_recebe_control': ifood_recebe_control,
                'custo_desconto_ifood': custo_desconto_ifood,
                 'lucro_ifood_target': ifood_recebe_target,
                'lucro_ifood_control': ifood_recebe_target,
                'roi_desconto_ifood_%': roi_desconto_ifood,
                'custo_por_conversao_adicional': custo_por_conversao_adicional,
                'ticket_medio_target': ticket_medio_target,
                'ticket_medio_control': ticket_medio_control,
                'conversoes_adicionais': conversoes_adicionais
            })
    
    return pd.DataFrame(resultados)

"""
📊 RELATÓRIO FINAL - TESTE A/B COM 20% DESCONTO

🎯 RESULTADOS ESTATÍSTICOS

CONVERSÃO (✓ ALTAMENTE SIGNIFICATIVO)
• Control: 24.12% dos clientes voltaram
• Target: 27.47% dos clientes voltaram  
• Lift: +13.9% (+3.404 clientes adicionais)
• Significância: p-value ≈ 0 (resultado REAL)

VALOR POR CLIENTE (⚠️ LIMITAÇÃO ESTATÍSTICA)
• Diferença: R$ 0,36 (target: R$ 20,58 vs control: R$ 20,22)
• Poder estatístico: 33.4% (BAIXO)
• MDE atual: R$ 0,37 (detecta apenas diferenças grandes)
• MDE para 80% poder: R$ 0,67 (necessário para confiança)

💰 VIABILIDADE FINANCEIRA (✓ POSITIVA)

RETORNO DO INVESTIMENTO
• Custo desconto: R$ 99.794,61
• Receita adicional: R$ 190.563,37
• Lucro líquido: R$ 90.768,76
• ROI: 91% (cada R$ 1 gerou R$ 0,91)

🎯 CONCLUSÕES E RECOMENDAÇÕES

✅ ESTRATÉGIA FUNCIONA PARA:
• Aumentar conversão para segunda compra
• Gerar retorno financeiro positivo
• Atrair mais clientes de volta

⚠️ PONTOS DE OTIMIZAÇÃO:
• Poder estatístico baixo para diferenças pequenas
• Desconto de 20% pode ser reduzido
• Testar janelas maiores que 5 dias

💡 PRÓXIMOS PASSOS SUGERIDOS:
1. Testar com 10-15% de desconto
2. Estender janela para 7-10 dias
3. Monitorar ticket médio dos convertidos
"""

Clientes com um unico pedido

In [ ]:
x = 1 # Exemplo: clientes com 2 pedidos

df_clientes_x_pedidos = (
    df[df['order_created_month'] == 12]
    .groupby(['customer_id', 'order_created_month'])
    .agg(
        quantidade_pedidos=('unique_order_hash', 'count')
    )
    .reset_index()
    .query(f'quantidade_pedidos == {x}')
)


In [ ]:
df_=df[['customer_id','order_created_month','merchant_id','order_created_at','days_until_next_order','rank_month','order_total_amount','amount_pct_change_from_previous','amount_diff_from_previous']]

In [ ]:
# 1. Valor das 2ªs ordens COM cupom
valor_segundas_ordens = df[df['rank_month'] == 2]['order_total_amount'].sum()

# 2. Custo total do cupom
custo_cupom = df[df['rank_month'] == 2]['%desconto'].sum()

# 3. Receita líquida das 2ªs ordens
receita_liquida = valor_segundas_ordens - custo_cupom

print("💰 IMPACTO DO CUPOM NA 2ª ORDEM:")
print(f"• Valor bruto das 2ªs ordens: R$ {valor_segundas_ordens:,.2f}")
print(f"• Custo do cupom: R$ {custo_cupom:,.2f}")
print(f"• Receita líquida: R$ {receita_liquida:,.2f}")
print(f"• % do custo sobre a receita: {(custo_cupom/valor_segundas_ordens*100):.1f}%")

In [ ]:


# Mostrar amostra
df[['customer_id', 'order_created_month','order_created_month' ,'rank_month', 'order_total_amount', '%desconto']].head(10)

In [ ]:
# Filtrar a base df apenas com os clientes que têm X pedidos
df_filtrado = df_[
    (df_['customer_id'].isin(df_clientes_x_pedidos['customer_id'])) #& 
    #(df['order_created_month'] == 12)
]
#df_filtrado.sort_values('amount_pct_change_from_previous')

In [ ]:
df_filtrado.head()

In [ ]:
from lifelines.statistics import logrank_test

# Supondo que você tem analysis_df com:
# - customer_id (identificador único)
# - grupo_ab ('A' ou 'B') 
# - tempo_ate_segunda (dias)
# - fez_segunda_ordem (True/False)

# Separar os grupos
grupo_a = clientes_unicos_primeiro_pedido[clientes_unicos_primeiro_pedido['is_target'] == 'target']
grupo_b = clientes_unicos_primeiro_pedido[clientes_unicos_primeiro_pedido['is_target'] == 'contol']

print(f"Grupo A: {len(grupo_a)} clientes")
print(f"Grupo B: {len(grupo_b)} clientes")

# Teste de Log-Rank
results = logrank_test(
    grupo_a['tempo_ate_segunda'], 
    grupo_b['tempo_ate_segunda'],
    event_observed_A=grupo_a['fez_segunda_ordem'],
    event_observed_B=grupo_b['fez_segunda_ordem']
)

print("\n=== TESTE DE LOG-RANK ===")
print(f"p-value: {results.p_value:.4f}")
print(f"Significativo (p < 0.05): {results.p_value < 0.05}")

if results.p_value < 0.05:
    print("✅ Há diferença significativa entre os grupos!")
else:
    print("❌ Não há diferença significativa entre os grupos")

In [ ]:
# Só precisa das contagens totais
from scipy.stats import chi2_contingency

# Criar tabela manualmente se não tiver o dataset
tabela = np.array([
    [len(grupo_a[grupo_a['fez_segunda_ordem'] == False]),  # Grupo A não converteu
     len(grupo_a[grupo_a['fez_segunda_ordem'] == True])],   # Grupo A converteu
    
    [len(grupo_b[grupo_b['fez_segunda_ordem'] == False]),  # Grupo B não converteu  
     len(grupo_b[grupo_b['fez_segunda_ordem'] == True])]    # Grupo B converteu
])

chi2, p_value, dof, expected = chi2_contingency(tabela)

In [ ]:
from scipy.stats import chi2_contingency

# Tabela de contingência
tabela = pd.crosstab(analysis_df['grupo_ab'], analysis_df['fez_segunda_ordem'])
print("=== TABELA DE CONTINGÊNCIA ===")
print(tabela)

# Teste Chi-Quadrado
chi2, p_value, dof, expected = chi2_contingency(tabela)
print(f"\n=== TESTE CHI-QUADRADO ===")
print(f"Chi2: {chi2:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Significativo (p < 0.05): {p_value < 0.05}")

In [ ]:
df.head()

In [ ]:
from datetime import datetime
data_inicio = datetime(2024, 1, 1)
data_inicio

In [ ]:
clientes_ativos_janeiro = df[df['order_created_month'] == 1]['customer_id'].unique()
df_clientes_jan = df[df['customer_id'].isin(clientes_ativos_janeiro)].copy()

In [ ]:
df=df[df['order_created_month'] == 1]

In [ ]:
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt

# Clientes ativos em janeiro de 2019 (fizeram pelo menos 1 pedido em jan/2019)
clientes_ativos_janeiro = df[
    (df['order_created_month'] == 1) & 
    (df['order_created_at'].dt.year == 2019)
]['customer_id'].unique()

print(f"Total de clientes ativos em janeiro de 2019: {len(clientes_ativos_janeiro)}")

# Filtrar TODOS os pedidos desses clientes
df_clientes_jan = df[df['customer_id'].isin(clientes_ativos_janeiro)].copy()

# Preparar dados para análise de sobrevivência CORRETA
analysis_data = []

for customer_id in clientes_ativos_janeiro:
    customer_orders = df_clientes_jan[df_clientes_jan['customer_id'] == customer_id].sort_values('order_created_at')
    
    # Primeira ordem do cliente em janeiro de 2019
    ordens_janeiro_2019 = customer_orders[
        (customer_orders['order_created_month'] == 1) & 
        (customer_orders['order_created_at'].dt.year == 2019)
    ]
    
    if len(ordens_janeiro_2019) >= 1:
        primeira_ordem_jan = ordens_janeiro_2019.iloc[0]
        data_primeira_ordem = primeira_ordem_jan['order_created_at']
        
        # Verificar se fez segunda ordem (em qualquer mês/ano)
        fez_segunda_ordem = len(customer_orders) >= 2
        
        if fez_segunda_ordem:
            segunda_ordem = customer_orders.iloc[1]
            data_segunda_ordem = segunda_ordem['order_created_at']
            
            # CORREÇÃO: Calcular tempo ENTRE 1ª e 2ª ordem
            tempo_ate_segunda = (data_segunda_ordem - data_primeira_ordem).days
            
            # Garantir que não seja negativo
            if tempo_ate_segunda < 0:
                print(f"AVISO: Tempo negativo para cliente {customer_id}")
                tempo_ate_segunda = 0
        else:
            # Cliente não fez segunda ordem - censurar após período razoável
            tempo_ate_segunda = 90  # Censurar em 90 dias
            fez_segunda_ordem = False
        
        analysis_data.append({
            'customer_id': customer_id,
            'grupo_ab': primeira_ordem_jan.get('grupo_ab', 'unknown'),
            'tempo_ate_segunda': tempo_ate_segunda,
            'fez_segunda_ordem': fez_segunda_ordem,
            'data_primeira_ordem': data_primeira_ordem
        })

# Criar DataFrame de análise
analysis_df = pd.DataFrame(analysis_data)

print(f"Clientes na análise: {len(analysis_df)}")
print(f"Clientes que fizeram 2ª ordem: {analysis_df['fez_segunda_ordem'].sum()}")
print(f"Taxa de conversão: {analysis_df['fez_segunda_ordem'].mean():.2%}")

# Verificar se há tempos negativos
if (analysis_df['tempo_ate_segunda'] < 0).any():
    print("⚠️  AVISO: Ainda há tempos negativos!")
    print(analysis_df[analysis_df['tempo_ate_segunda'] < 0])

# Aplicar Kaplan-Meier
kmf = KaplanMeierFitter()
kmf.fit(
    durations=analysis_df['tempo_ate_segunda'],
    event_observed=analysis_df['fez_segunda_ordem']
)

# Plotar
plt.figure(figsize=(12, 8))
kmf.plot_survival_function()
plt.title('Tempo entre 1ª e 2ª Ordem\n(Clientes Janeiro 2019)')
plt.ylabel('Probabilidade de Ainda Não Ter Feito 2ª Ordem')
plt.xlabel('Dias desde a Primeira Ordem')
plt.grid(True)
plt.show()

# Estatísticas
print(f"\n--- Estatísticas ---")
print(f"Mediana de tempo até 2ª ordem: {kmf.median_survival_time_} dias")
for dias in [1, 3, 7, 15, 30]:
    conversao = (1 - kmf.predict(dias)) * 100
    print(f"Conversão em {dias} dias: {conversao:.1f}%")

In [ ]:
second_orders = df_f[df_f['rank_month'] == 2]

In [ ]:
df_f

In [ ]:
df_g = (
        second_orders.groupby(['order_created_month','days_since_first_order_month'])
          .agg(
              total_clientes=('customer_id', 'nunique'),
              total_pedidos=('order_id', 'count'),
              total_ordem=('order_total_amount', 'sum')
          )
          .reset_index()
          .assign(
              perc_clientes=lambda d: d.groupby('order_created_month')['total_clientes']
                                         .transform(lambda x: x / x.sum() * 100),
              perc_pedidos=lambda d: d.groupby('order_created_month')['total_pedidos']
                                        .transform(lambda x: x / x.sum() * 100),
              perc_amount=lambda d: d.groupby('order_created_month')['total_ordem']
                                         .transform(lambda x: x / x.sum() * 100)
          )
          .round(2)
          .sort_values(['order_created_month', 'days_since_first_order_month'], ascending=[True, True])
          .reset_index(drop=True)
    )
df_g[df_g['order_created_month']==12]
#df_g[df_g['order_created_month']==12]['total_ordem'].sum()


In [ ]:
df.head()

In [ ]:
gerar_stats(df,['day']).head()

In [ ]:
import streamlit as st

df.to_parquet(BASE_PATH / "gold" / "df_app.parquet", index=False)
estado = st.selectbox("Escolha o estado", df["weekday"].unique())
#mes = st.selectbox("Escolha o mês", df["month"].unique())

df_filtrado = df[(df["weekday"] == estado) #& (df["month"] == mes)
                 ]
st.dataframe(df_filtrado)


In [ ]:
df_pub_u=process_orders_pandas(df_pub_u)

In [ ]:
df_pub_un = df_pub_u[["customer_id","is_target", "order_created_month", "num_pedidos_mes", "num_pedidos_hist",'total_amount_mes','ticket_medio']].drop_duplicates().reset_index(drop=True)
df_pub_un['pedidos_sum'] = np.where(
        df_pub_un['num_pedidos_mes'] > 10, '10+',
        df_pub_un['num_pedidos_mes'].astype(str)
    )
df_pub_un.head()

In [ ]:
df_pub_un = df[["customer_id","is_target", "order_created_month", "num_pedidos_mes", "num_pedidos_hist",'total_amount_mes','ticket_medio']].drop_duplicates().reset_index(drop=True)
df_pub_un['pedidos_sum'] = np.where(
        df_pub_un['num_pedidos_mes'] > 10, '10+',
        df_pub_un['num_pedidos_mes'].astype(str)
    )
df_pub_un.head()